<a href="https://colab.research.google.com/github/LasTAD/pysyft_experiments/blob/main/PySyft_Exp_DS_small_signal_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подготовка пакета PySyft.

In [2]:
import sys
if "google.colab" in sys.modules:
    branch = "master"    # change to the branch you want
    ! git clone --single-branch --branch $branch https://github.com/OpenMined/PySyft.git
    ! cd PySyft && ./scripts/colab.sh      # fixes some colab python issues
    sys.path.append("/content/PySyft/src") # prevents needing restart

Cloning into 'PySyft'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 48409 (delta 0), reused 2 (delta 0), pack-reused 48407
Receiving objects: 100% (48409/48409), 55.91 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (32540/32540), done.
> git pull
> using latest commit
  15355b7f Delete stale.yml
> installing pip requirements
> patching python 3.6.9 colab bug
sed: can't read /usr/local/lib/python3.6/dist-packages/typeguard/__init__.py: No such file or directory
> colab environment ready


Подключение к владельцу данных.

In [1]:
import syft as sy
duet = sy.join_duet("84cf1e4676903cb28e19440a74e7aa35")
sy.logging(file_path="./syft_ds.log")

ModuleNotFoundError: ignored

In [ ]:
import torch
rem_torch = duet.torch

NameError: ignored

Параметры

In [ ]:
args = {
    "batch_size": 64,
    "log_batch_size": 1000,
    "epochs": 10,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
}

Список доступных тензоров

In [ ]:
duet.store.pandas

Загрузим удаленные данные

In [ ]:
X_train = duet.store["X_train"]
y_train = duet.store["y_train"]
train_num = duet.store["train_num"]

In [ ]:
class DatasetFromPointer(data.Dataset):
    def __init__(self, 
                 X_tensorpointer,
                 y_tensorpointer,
                 datanum_pointer,
                 ):
        super(DatasetFromPointer, self).__init__()
        self.X_tensorpointer = X_tensorpointer
        self.y_tensorpointer = y_tensorpointer
        self.datanum_pointer = datanum_pointer

    def __getitem__(self, index):
        input = self.X_tensorpointer[index]
        target = self.y_tensorpointer[index]
        return input, target

    def __len__(self):
        return self.datanum_pointer.get(
        request_block=True,
        reason="To write the training loop",
        timeout_secs=30,
        delete_obj=False,
    )
        
def batch_idx_fn(batch):
    return batch[0]

In [ ]:
train_set = DatasetFromPointer(X_train, y_train, train_num)
training_data_loader = DataLoader(dataset=train_set, batch_size=args["batchSize"], shuffle=True, collate_fn=batch_idx_fn)

Зададим нашу сеть


In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.lin1 = self.torch_ref.nn.Linear(X_train.shape[1],X_train.shape[1]*2)
        self.lin2 = self.torch_ref.nn.Linear(X_train.shape[1]*2, 1)
       
    def forward(self, x):
        x = self.torch_ref.nn.functional.relu(self.lin1(x))
        output = self.torch_ref.nn.functional.sigmoid(self.lin2(x))
        return output

In [ ]:
local_net = SyNet(rem_torch)
remote_net = model.send(duet)

Проверка CUDA

In [ ]:
has_cuda = False
has_cuda_ptr = rem_torch.cuda.is_available()
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    name="cuda_is_available",
    reason="To run test and inference locally",
    timeout_secs=5,  # change to something slower
))
print(has_cuda)

use_cuda = not args["no_cuda"] and has_cuda
# now we can set the seed
rem_torch.manual_seed(args["seed"])

device = rem_torch.device("cuda" if use_cuda else "cpu")
print(f"Data Owner device is {device.type.get()}")

In [ ]:
# if we have CUDA lets send our model to the GPU
if has_cuda:
    model.cuda(device)
else:
    model.cpu()

Обучение модели

In [ ]:
params = model.parameters()
optimizer = rem_torch.optim.SGD(params, lr=args["lr"])
scheduler = rem_torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args["gamma"])
criterion = rem_torch.nn.functional.binary_cross_entropy()

In [ ]:
for epoch in range(args["epochs"]):
    
    remote_net.train()
    epoch_loss = 0
    for batch_idx, data_pointers in enumerate(training_data_loader):

        optimizer.zero_grad()
        data_ptr, target_ptr = data_pointers[0], data_pointers[1]
        data_ptr_reshape = remote_torch.unsqueeze(remote_torch.unsqueeze(data_ptr, 0), 0)
        target_ptr_reshape = remote_torch.unsqueeze(remote_torch.unsqueeze(target_ptr, 0), 0)
       
        output_ptr = remote_net(data_ptr_reshape)
        
        loss = criterion(output_ptr, target_ptr_reshape)
        loss.backward()
        optimizer.step()

        if batch_idx % args["log_batch_size"] == 0:
            loss_item = loss.item().get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=3,
                delete_obj=False,
                verbose=False
                )
            print(f"epoch {epoch}, batch_idx {batch_idx}, loss {loss_item}")

        if args["dry_run"]:
            break

Сохранить модель

In [ ]:
remote_net.get(
    request_block=True,
    reason="test evaluation",
    timeout_secs=5
).save(f"super_resolve.pt")